In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os
os.getcwd()

'/content'

In [8]:
path="/content/drive/MyDrive/Colab_Notebooks/panda_grasp/"
os.chdir(path)
os.getcwd()

'/content/drive/MyDrive/Colab_Notebooks/panda_grasp'

In [11]:
!ls

callback.py	  mainTest.ipynb    __pycache__		       test2.py
core.py		  panda.py	    robot		       test.py
custom_policy.py  PandaReachEnv.py  script_test.py	       wrapper.py
__init__.py	  pyBullet.py	    stable_baseline_helper.py


In [10]:
cd Environment/

/content/drive/My Drive/Colab_Notebooks/panda_grasp/Environment


In [ ]:
pip install stable-baselines3[extra]

In [ ]:
pip install gym

In [4]:
pip install pybullet

     |████████████████████████████████| 79.0MB 44kB/s 
  Created wheel for pybullet: filename=pybullet-3.1.7-cp37-cp37m-linux_x86_64.whl size=89749544 sha256=282b41a498119b46ec079fff77f7caa8ad0cfda7fbd1c74725b3992e74533425
  Stored in directory: /root/.cache/pip/wheels/30/56/e6/fce8276a2f30165f7ac31089bb72f390fa16b87328651e1a5a
Successfully built pybullet


In [1]:
from stable_baselines3.ppo.ppo import PPO
import torch.nn as nn
import gym
import os
from panda import PandaEnv
from bullet.pyBullet import PyBullet
from task.Grasp.PandaGraspEnv import PandaGraspEnv
from task.Reach.PandaReachEnv import PandaReachEnv
from task.wrapper import ProcessDepthFrame84, ProcessFrame84, DepthToPyTorch, ImageToPyTorch, MoveConstraint
# from PandaReachEnv import PandaReachEnv
import matplotlib.pyplot as plt
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_checker import check_env
import numpy as np
from custom_policy import CustomCNN, CustomActorCriticPolicy, ResNetNetwork
from stable_baselines3.common.evaluation import evaluate_policy
from callback import CheckpointCallback, EvalCallback, SaveVecNormalizeCallback

In [2]:
env = PandaGraspEnv(sim = PyBullet(render =False))

env = ProcessFrame84(env)
env = ImageToPyTorch(env)
env = MoveConstraint(env)
'''
env = ProcessDepthFrame84(env)
# env = DepthToPyTorch(env)
'''
obs = env.reset()

print(obs.shape)


(3, 64, 64)


In [3]:
import matplotlib.pyplot as plt
import numpy as np
plt.imshow(obs)

TypeError: Invalid shape (3, 64, 64) for image data

In [3]:
policy_kwargs = dict(
    features_extractor_class=ResNetNetwork,
    features_extractor_kwargs=dict(features_dim=64),
)
checkpoint_callback = CheckpointCallback(save_freq=1000, save_path='./log/',
                                         name_prefix='rl_model')
save_vec_normalize = SaveVecNormalizeCallback(
                save_freq=1, save_path='./log/Vec/')
eval_callback = EvalCallback(
                eval_env=env,
                callback_on_new_best=save_vec_normalize,
                best_model_save_path='./log/best_model/',
                n_eval_episodes=5,
                log_path='./log/Eval/',
                eval_freq=10000,
                deterministic=True,
            )

In [4]:
check_env(env)

/home/yi/miniconda3/envs/drl_env/lib/python3.7/site-packages/stable_baselines3/common/env_checker.py:26: UserWarning: It seems that your observation is an image but the `dtype` of your observation_space is not `np.uint8`. If your observation is not an image, we recommend you to flatten the observation to have only a 1D vector
  "It seems that your observation is an image but the `dtype` "
/home/yi/miniconda3/envs/drl_env/lib/python3.7/site-packages/stable_baselines3/common/env_checker.py:42: UserWarning: The minimal resolution for an image is 36x36 for the default CnnPolicy. You might need to use a custom `cnn_extractor` cf https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html
  "The minimal resolution for an image is 36x36 for the default CnnPolicy. "



In [5]:
model = PPO('MlpPolicy', env=env, verbose=1, policy_kwargs=policy_kwargs)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
================ resnet18
Using cache found in /home/yi/.cache/torch/hub/pytorch_vision_v0.9.0
Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1)

In [6]:
model.learn(200000, callback = checkpoint_callback)
print("Learning Finished")


KeyboardInterrupt: 